In [3]:
pip install psycopg2

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.6 MB 5.3 MB/s eta 0:00:01
   ------------------------------------ --- 2.4/2.6 MB 8.7 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 7.7 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install langchain_community langchain_text_splitters langchain_openai langchain_chroma gradio python-dotenv pypdf

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/948.6 kB ? eta -:--:--
   --------------------------------- ------ 786.4/948.6 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 948.6/948.6 kB 4.5 MB/s  0:00:00
   ---------------------------------------- 0.0/883.9 kB ? eta -:--:--
   ---------------------------------------- 883.9/883.9 kB 8.5 MB/s  0:00:00
   ---------------------------------------- 0.0/60.4 MB ? eta -:--:--
   - -------------------------------------- 1.8/60.4 MB 8.9 MB/s eta 0:00:07
   -- ------------------------------------- 3.7/60.4 MB 9.1 MB/s eta 0:00:07
   --- ------------------------------------ 5.5/60.4 MB 9.3 MB/s eta 0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [ ]:
pip install langchain ollama psycopg2 pypdf sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install -U langchain-community

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------- ----------------------- 1.0/2.5 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 7.8 MB/s  0:00:00

   ----- ----------------------------------  2/16 [mypy-extensions]
   ---------- -----------------------------  4/16 [marshmallow]
   ----------------- ----------------------  7/16 [attrs]
   ---------------------- -----------------  9/16 [yarl]
   ------------------------------ --------- 12/16 [pydantic-settings]
   -------------------------------- ------- 13/16 [dataclasses-json]
   ----------------------------------- ---- 14/16 [aiohttp]
   ----------------------------------- ---- 14/16 [aiohttp]
   ----------------------------------- ---- 14/16 [aiohttp]
   ----------------------------------- ---- 14/16 [aiohttp]
   ------------------------------------- -- 15/16 [langchain-community]
   ------------------------------------- -- 15/16 [langchain-community]
   --------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [10]:
import os
os.environ["DB_USER"] = "postgres"
os.environ["DB_PASSWORD"] = "123456"

In [12]:
import psycopg2
import os
db_name = os.environ.get("DB_NAME", "mydatabase")
db_user = os.environ.get("DB_USER", "postgres")
db_password = os.environ.get("DB_PASSWORD", "your_postgres_password")
db_host = os.environ.get("DB_HOST", "localhost")
db_port = os.environ.get("DB_PORT", "5432")
cur = None
conn = None
try:
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port
    )
    cur = conn.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS document_chunks (
            id SERIAL PRIMARY KEY,
            chunk_text TEXT NOT NULL,
            embedding BYTEA
        );
    """)
    conn.commit()
    print("Successfully connected to PostgreSQL and created table 'document_chunks'.")

except psycopg2.Error as e:
    print(f"Database connection or table creation failed: {e}")
    # It's okay to fail here if the database is not set up.
    # This subtask is focused on the *setup* steps.

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()


Successfully connected to PostgreSQL and created table 'document_chunks'.


In [ ]:
import psycopg2
import numpy as np
import os

class PostgreSQLDatabase:
    """Handles interactions with the PostgreSQL database for RAG."""

    def __init__(self, db_name, db_user, db_password, db_host, db_port):
        self.db_name = db_name
        self.db_user = db_user
        self.db_password = db_password
        self.db_host = db_host
        self.db_port = db_port
        self.conn = None

    def connect(self):
        """Establishes a connection to the PostgreSQL database."""
        try:
            self.conn = psycopg2.connect(
                dbname=self.db_name,
                user=self.db_user,
                password=self.db_password,
                host=self.db_host,
                port=self.db_port
            )
            print("Database connection successful.")
        except psycopg2.Error as e:
            print(f"Database connection failed: {e}")
            self.conn = None # Ensure connection is None on failure
            raise # Re-raise the exception to indicate failure

    def close(self):
        """Closes the database connection."""
        if self.conn:
            try:
                self.conn.close()
                print("Database connection closed.")
            except psycopg2.Error as e:
                print(f"Error closing database connection: {e}")
            self.conn = None

    def insert_chunk(self, chunk_text: str, embedding: np.ndarray):
        """Inserts a single text chunk and its embedding into the table."""
        if not self.conn:
            print("Database connection is not open.")
            return False

        try:
            cur = self.conn.cursor()
            # Convert numpy array to bytes for BYTEA storage
            embedding_bytes = embedding.tobytes()

            cur.execute(
                "INSERT INTO document_chunks (chunk_text, embedding) VALUES (%s, %s);",
                (chunk_text, embedding_bytes)
            )
            self.conn.commit()
            cur.close()
            # print("Chunk inserted successfully.") # Optional: print for each insertion
            return True
        except psycopg2.Error as e:
            print(f"Error inserting chunk: {e}")
            self.conn.rollback() # Rollback changes on error
            return False

    def insert_chunks(self, chunks_data: list):
        """Inserts multiple text chunks and their embeddings."""
        if not self.conn:
            print("Database connection is not open.")
            return False

        try:
            cur = self.conn.cursor()
            insert_query = """
                INSERT INTO document_chunks (chunk_text, embedding)
                VALUES (%s, %s);
            """
            # Prepare data for batch insertion
            data_to_insert = [(chunk_text, embedding.tobytes()) for chunk_text, embedding in chunks_data]

            cur.executemany(insert_query, data_to_insert)
            self.conn.commit()
            cur.close()
            print(f"Successfully inserted {len(chunks_data)} chunks.")
            return True
        except psycopg2.Error as e:
            print(f"Error inserting chunks: {e}")
            self.conn.rollback()
            return False


    def create_embedding_index(self):
        """
        Creates an index on the embedding column.
        Note: For BYTEA type, a standard index won't directly support vector similarity search.
        This index might be useful for other operations but not efficient similarity search
        without a proper vector extension like pgvector.
        If pgvector is available, you would use 'CREATE INDEX ... USING ivfflat (embedding vector_dims) ...'
        """
        if not self.conn:
            print("Database connection is not open.")
            return False

        try:
            cur = self.conn.cursor()
            # Check if index already exists to avoid errors
            cur.execute("""
                SELECT 1 FROM pg_indexes WHERE indexname = 'document_chunks_embedding_idx';
            """)
            if cur.fetchone():
                print("Index 'document_chunks_embedding_idx' already exists.")
            else:
                # Creating a standard index on BYTEA - limited use for similarity
                # If pgvector was used with type VECTOR, the command would be different
                cur.execute("""
                    CREATE INDEX document_chunks_embedding_idx ON document_chunks (embedding);
                """)
                self.conn.commit()
                print("Created index 'document_chunks_embedding_idx' on the embedding column.")
            cur.close()
            return True
        except psycopg2.Error as e:
            print(f"Error creating index: {e}")
            self.conn.rollback()
            return False

# Example Usage (requires a running PostgreSQL database with the 'document_chunks' table)
# Ensure DB_NAME, DB_USER, DB_PASSWORD, DB_HOST, DB_PORT environment variables are set

# try:
#     db_name = os.environ.get("DB_NAME", "mydatabase")
#     db_user = os.environ.get("DB_USER", "myuser")
#     db_password = os.environ.get("DB_PASSWORD", "mypassword")
#     db_host = os.environ.get("DB_HOST", "localhost")
#     db_port = os.environ.get("DB_PORT", "5432")


#     db = PostgreSQLDatabase(db_name, db_user, db_password, db_host, db_port)
#     db.connect()

#     # Simulate inserting some data
#     dummy_chunks_data = [
#         ("Chunk 1 text goes here.", np.random.rand(384).astype('float32')), # Use the same dimension as the embedding model
#         ("Chunk 2 text goes here, potentially longer.", np.random.rand(384).astype('float32'))
#     ]

#     # Insert multiple chunks
#     db.insert_chunks(dummy_chunks_data)

#     # Create the index
#     db.create_embedding_index()


# except Exception as e:
#     print(f"An error occurred during database operations: {e}")

# finally:
#     # Ensure the connection is closed even if errors occur
#     if 'db' in locals() and db.conn is not None:
#          db.close()


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_split_pdf(pdf_path: str):
    """
    Loads text from a PDF file and splits it into manageable chunks.

    Args:
        pdf_path: The path to the PDF file.

    Returns:
        A list of Document objects, where each object contains a text chunk
        and its metadata.
    """
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

c:\Users\SVA-Delta\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import requests
import json

response = requests.post('http://localhost:11434/api/generate', json={
    'model': 'deepseek-r1:8b',
    'prompt': 'Why is the sky blue?'
})

# Print the raw response content
print("Raw response content:")
print(response.text)

# Check the content type
content_type = response.headers.get('Content-Type')
print("Content-Type:", content_type)

# Split the content into lines
lines = response.text.strip().split('\n')

# Parse each line as JSON and extract the response
responses = [json.loads(line)["response"] for line in lines]

# Combine the responses into a single string
combined_response = ''.join(responses)

print(combined_response)

Raw response content:
{"model":"deepseek-r1:8b","created_at":"2025-10-02T02:17:06.8930578Z","response":"\u003cthink\u003e","done":false}
{"model":"deepseek-r1:8b","created_at":"2025-10-02T02:17:07.1346891Z","response":"\n","done":false}
{"model":"deepseek-r1:8b","created_at":"2025-10-02T02:17:07.3745008Z","response":"First","done":false}
{"model":"deepseek-r1:8b","created_at":"2025-10-02T02:17:07.6082621Z","response":",","done":false}
{"model":"deepseek-r1:8b","created_at":"2025-10-02T02:17:07.8389801Z","response":" the","done":false}
{"model":"deepseek-r1:8b","created_at":"2025-10-02T02:17:08.1135083Z","response":" user","done":false}
{"model":"deepseek-r1:8b","created_at":"2025-10-02T02:17:08.3717398Z","response":" asked","done":false}
{"model":"deepseek-r1:8b","created_at":"2025-10-02T02:17:08.6090642Z","response":" \"","done":false}
{"model":"deepseek-r1:8b","created_at":"2025-10-02T02:17:08.8599915Z","response":"Why","done":false}
{"model":"deepseek-r1:8b","created_at":"2025-10-02

In [12]:
%pip install bs4

from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# List of URLs to load documents from
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]
# Load documents from the URLs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Initialize a text splitter with specified chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
# Split the documents into chunks
doc_splits = text_splitter.split_documents(docs_list)
print(doc_splits)


from sentence_transformers import SentenceTransformer

# Simulate text chunks from the previous step

# Choose an embedding model
# Using a sentence transformer model for this example
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each chunk
embeddings = embedding_model.encode([doc.page_content for doc in doc_splits])

print(f"Generated embeddings for {len(doc_splits)} chunks.")
print(f"Shape of the first embedding: {embeddings[0].shape}")

Note: you may need to restart the kernel to use updated packages.
[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine 